# Apartado 1: Particionado

In [1]:
from Datos import Datos
from ValidacionBootstrap import ValidacionBootstrap
from ValidacionCruzada import ValidacionCruzada
from ValidacionSimple import ValidacionSimple

balloons = Datos('ConjuntosDatos/balloons.data')
tic_tac_toe = Datos('ConjuntosDatos/tic-tac-toe.data')

validaciones = [ValidacionSimple(70), ValidacionCruzada(6), ValidacionBootstrap()]

for val in validaciones:
    val.creaParticiones(tic_tac_toe.datos)
    
    print("\nÍndices de ", val.nombre_estrategia, " para tic-tac-toe: ", sep='')
    for particion in val.particiones:
        print("\tTrain: ", sorted(particion.indicesTrain)[:20], "...", sep='')
        print("\tTest: ", sorted(particion.indicesTest)[:20], "...", sep='')

print("\n\n")
for val in validaciones:
    val.creaParticiones(balloons.datos)
    
    print("\nÍndices de ", val.nombre_estrategia, " para balloons: ", sep='')
    for particion in val.particiones:
        print("\tTrain: ", sorted(particion.indicesTrain), sep='')
        print("\tTest: ", sorted(particion.indicesTest), sep='')


FileNotFoundError: [Errno 2] No such file or directory: 'ConjuntosDatos/balloons.data'

## Descripcción de los índices de train y test

Hemos imprimido los índices ordenados para que sean más fáciles de analizar a simple vista.

Vemos que para validación simple, cogemos todos los índices y aleatoriamente tomamos un porcentaje para test y el resto son de test.

Para validación cruzados vemos cómo se crean las 6 particiones (folds) que hemos especificado, y que en cada una de las particiones los índices de test son diferentes hasta completar todos los índices. Los índices de train de cada partición son los que no hemos tomado para test.

En validación por Bootstrap tomamos como train tantos índices aleatorios como datos haya, permitiendo que haya repeticiones, y los datos de test son aquellos índices que no hemos cogido.


## Ventajas/desventajas de cada una de las validaciones

Validación simple es claramente el más fácil de entender e implementar y si se dividen los datos aleatoriamente funciona bien.

Validación cruzada al hacer más particiones dará una tasa de fallos más realista, pero a costa de tener que realizar múltiples veces el proceso de entrenamiento, haciendo que el tiempo de validación aumente proporcionalmente al número de folds.

El principal inconveniente de Bootstrap es que elegimos para train varias veces los mismos ejemplos, haciendo que la tasa de error esté subestimada.

# Apartado 2: Naive-Bayes

In [3]:
import numpy as np

from ClasificadorNaiveBayes import ClasificadorNaiveBayes

german = Datos('ConjuntosDatos/german.data')
nb = ClasificadorNaiveBayes()

for val in validaciones:
    format_header = "\t%-30s %-30s %-30s"
    format_cell = "\t%-30s %-30f %-30f"
    
    print("\n\n", val.nombre_estrategia, "sin corrección de Laplace:")
    print(format_header % ("", "tasa de error", "desviación típica del error"))
    
    errores, tasa_de_error = nb.validacion(val, balloons, nb, aplicar_correccion_de_laplace=False)
    print(format_cell % ("balloons", tasa_de_error, np.std(errores)))
    
    errores, tasa_de_error = nb.validacion(val, tic_tac_toe, nb, aplicar_correccion_de_laplace=False)
    print(format_cell % ("tic-tac-toe", tasa_de_error, np.std(errores)))
    
    errores, tasa_de_error = nb.validacion(val, german, nb, aplicar_correccion_de_laplace=False)
    print(format_cell % ("german", tasa_de_error, np.std(errores)))
    
    
    print("\n", val.nombre_estrategia, "con corrección de Laplace:")
    print(format_header % ("", "tasa de error", "desviación típica del error"))
    
    errores, tasa_de_error = nb.validacion(val, balloons, nb)
    print(format_cell % ("balloons", tasa_de_error, np.std(errores)))
    
    errores, tasa_de_error = nb.validacion(val, tic_tac_toe, nb)
    print(format_cell % ("tic-tac-toe", tasa_de_error, np.std(errores)))
    
    errores, tasa_de_error = nb.validacion(val, german, nb)
    print(format_cell % ("german", tasa_de_error, np.std(errores)))


ModuleNotFoundError: No module named 'ClasificadorNaiveBayes'

## Análisis de los resultados

Vemos que para balloons la tasa de fallos es muy baja o nula. Esto es debido a que el número de datos es muy pequeño.

Para tic-tac-toe y German vemos que con validación simple al aplicar la corrección de Laplace la tasa de aciertos mejora, mientras que para validación cruzada permanece prácticamente igual y para bootstrap empeora.

En tic-tac-toe vemos que el porcentaje de errores para cualquiera de las validaciones está en torno a un 30%

En los datos de German el porcentaje de errores es ligeramente menor, estando en torno a un 25%

# Apartado 3: Scikit-Learn

In [4]:
import itertools

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB

for dataset in [german, tic_tac_toe]:
    if dataset == german:
        print("german:")
    else:
        print("tic-tac-toe:")
    
    # Encode categorical integer features using a one-hot aka one-of-K scheme (categorical features). Hay DeprecationWarnings
    encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1], sparse=False)
    X = encAtributos.fit_transform(dataset.datos[:, :-1])
    
    # Clases correspondientes a cada uno de los array de X
    y = dataset.datos[:, -1]
    
    # Particiones del modo Validacion Simple
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # Entrenamiento y clasificacion de X_test
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    
    errores = y_pred != y_test
    tasa_de_error = sum(errores) / len(errores)
    
    print("\tTasa de error con validación simple:", tasa_de_error)
    
    # Particiones del modo Validacion Cruzada
    kf = KFold(n_splits=6, shuffle=True)
    
    errores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        
        errores.append(y_pred != y_test)
    
    errores = list(itertools.chain.from_iterable(errores))
    tasa_de_error = sum(errores) / len(errores)
    print("\tTasa de error con validación cruzada:", tasa_de_error)

print("\n" * 2)


NameError: name 'german' is not defined

Hemos hecho la mismas validaciones que en el apartado anterior y vemos que los porcentajes de error son muy similares a los que teníamos antes